<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/llm/openvino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# OpenVINO 语言模型

[OpenVINO™](https://github.com/openvinotoolkit/openvino) 是一个用于优化和部署AI推断的开源工具包。OpenVINO™ Runtime 可以在各种硬件[设备](https://github.com/openvinotoolkit/openvino?tab=readme-ov-file#supported-hardware-matrix)上运行经过优化的相同模型。加速您在语言模型 + LLMs、计算机视觉、自动语音识别等用例中的深度学习性能。

通过 `OpenVINOLLM` 实体封装的 LlamaIndex，可以在本地运行 OpenVINO 模型：


在下面的代码中，我们安装了这个演示所需的包：


In [ ]:
%pip install llama-index-llms-openvino transformers huggingface_hub

现在我们已经准备好了，让我们开始玩一下吧：


如果您在Colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
!pip install llama-index

In [ ]:
from llama_index.llms.openvino import OpenVINOLLM

In [ ]:
def messages_to_prompt(messages):    prompt = ""    for message in messages:        if message.role == "system":            prompt += f"<|system|>\n{message.content}</s>\n"        elif message.role == "user":            prompt += f"<|user|>\n{message.content}</s>\n"        elif message.role == "assistant":            prompt += f"<|assistant|>\n{message.content}</s>\n"    # 确保我们以系统提示开始，如果需要则插入空白    if not prompt.startswith("<|system|>\n"):        prompt = "<|system|>\n</s>\n" + prompt    # 添加最终的助手提示    prompt = prompt + "<|assistant|>\n"    return promptdef completion_to_prompt(completion):    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"

### 模型加载

可以通过使用 `OpenVINOLLM` 方法来指定模型参数来加载模型。

如果你有英特尔GPU，可以指定 `device_map="gpu"` 来在其上运行推断。


In [ ]:
ov_config = {
    "PERFORMANCE_HINT": "LATENCY",
    "NUM_STREAMS": "1",
    "CACHE_DIR": "",
}

ov_llm = OpenVINOLLM(
    model_name="HuggingFaceH4/zephyr-7b-beta",
    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"ov_config": ov_config},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="cpu",
)

In [ ]:
response = ov_llm.complete("What is the meaning of life?")
print(str(response))

### 使用本地OpenVINO模型进行推理

可以使用命令行界面将您的模型[导出](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#export)为OpenVINO IR格式，并从本地文件夹加载模型。


In [ ]:
!optimum-cli export openvino --model HuggingFaceH4/zephyr-7b-beta ov_model_dir

建议使用`--weight-format`对权重进行8位或4位量化，以减少推理延迟和模型占用空间。


In [ ]:
!optimum-cli export openvino --model HuggingFaceH4/zephyr-7b-beta --weight-format int8 ov_model_dir

In [ ]:
!optimum-cli export openvino --model HuggingFaceH4/zephyr-7b-beta --weight-format int4 ov_model_dir

In [ ]:
ov_llm = OpenVINOLLM(
    model_name="ov_model_dir",
    tokenizer_name="ov_model_dir",
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"ov_config": ov_config},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="gpu",
)

您可以通过激活的动态量化和KV-cache量化来获得额外的推理速度提升。可以通过以下方式在`ov_config`中启用这些选项：


In [ ]:
ov_config = {
    "KV_CACHE_PRECISION": "u8",
    "DYNAMIC_QUANTIZATION_GROUP_SIZE": "32",
    "PERFORMANCE_HINT": "LATENCY",
    "NUM_STREAMS": "1",
    "CACHE_DIR": "",
}

### 数据流

使用 `stream_complete` 终端点


In [ ]:
response = ov_llm.stream_complete("Who is Paul Graham?")
for r in response:
    print(r.delta, end="")

使用 `stream_chat` 端点


In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = ov_llm.stream_chat(messages)

for r in resp:
    print(r.delta, end="")

有关更多信息，请参考：

* [OpenVINO LLM指南](https://docs.openvino.ai/2024/learn-openvino/llm_inference_guide.html)。

* [OpenVINO文档](https://docs.openvino.ai/2024/home.html)。

* [OpenVINO入门指南](https://www.intel.com/content/www/us/en/content-details/819067/openvino-get-started-guide.html)。
